In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

In [3]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [4]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [5]:
df = df.dropna()

In [6]:
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)

In [9]:
!pip install geocoder
import geocoder

In [11]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
postal_codes = df['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [16]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.72321,-79.45141
4,M4A,North York,Victoria Village,43.66449,-79.39302
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66277,-79.52831
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.81153,-79.19552
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.74929,-79.36169


In [17]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.72321,-79.45141
4,M4A,North York,Victoria Village,43.66449,-79.39302
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66277,-79.52831
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.81153,-79.19552
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.74929,-79.36169


In [18]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.72321,-79.45141
4,M4A,North York,Victoria Village,43.66449,-79.39302
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66277,-79.52831
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.81153,-79.19552
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.74929,-79.36169
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.65736,-79.37818
10,M1B,Scarborough,"Malvern, Rouge",43.70799,-79.44838
12,M3B,North York,Don Mills,43.78564,-79.15871
13,M4B,East York,"Parkview Hill, Woodbine Gardens",43.72184,-79.34340
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.68970,-79.30680
